# Assessment 2 MATH70094 Programming for Data Science Autumn 2025

# Enter your NAME and CID here

# Question 2 (16 marks)

In this question you continue the analysis of the flight data using Python. 
 
Some helpful functions for this question are `merge`, `apply`, `groupby`, `dropna`.

## Code clarity (1 mark)

There is a famous saying among software developers that code is read more often than it is written. Marks will be awarded (or not awarded) based on the clarity of the code and appropriate use of comments.

## Part A

### Part A(i) (1 mark)

Load the contents of the csv files `flights.csv` and `planes.csv` as dataframes named `flights` and `planes`. Print the first 6 rows from each dataframe. How many flights are there in all?

### Part A(ii) (1 mark)

Find the number of missing entries for each column in `flights`.

### Part A(iii) (2 marks)

Add a new column named `long_delayed_AA` to `flights`, which contains for a flight the value `True` if it is a flight by the carrier 'AA', is long haul (`distance` more than 1000 miles) and arrived with more than 30 minutes delay. Otherwise, it contains the value `False`. To check the results after adding the column, print the first 10 rows of `flights` including only the columns `carrier`, `distance`, `arr_delay` and `long_delayed_AA`. For how many flights is `long_delayed_AA` equal to `True`?  

## Part B

There are many missing entries in the `arr_delay` column in `flights`. Your task is to impute as many missing values using information from other variables.

### Part B(i) (3 marks)

To identify which covariates affect `arr_delay`, create three dataframes from `flights` as follows:
- a dataframe named `monthly_stats` with two columns, which in each row contains for one of the 12 months the average arrival delay in that month as well as the total number of flights,
- a dataframe named `hourly_stats` with two columns, which in each row contains for one of the 24 departure hours in a day the average arrival delay in that departure hour as well as the total number of flights,
- a dataframe named `carrier_stats` with two columns, which in each row contains for one of the carriers the average arrival delay for that carrier as well as the total number of flights. 

Print all three newly created dataframes. Describe what you notice for each dataframe using one sentence.

### Part B(iii) (2 marks)

Create a new dataframe `flights_imp` from `flights`. In the new dataframe, impute the missing `arr_delay` entries. To do so, fill in for a given row with a missing `arr_delay` entry the arrival delay averaged over all the flights with the same carrier within the same month and the same hour. After imputation, how many missing entries are left in the column `arr_delay`?

### Part B(iii) (1 mark)

Using the `describe` method from Pandas, discuss if the distribution (quantiles, min, max) of the values in `arr_delay` has changed after imputation.

## Part C

You are asked to study the size of planes appearing in `flights`. 

### Part C(i) (1 mark)

Add a new column named `size_category` to `planes` using the `apply` function from Pandas. The new column should indicate if a plane is `Small` (less than 100 seats), `Medium` (between 100 and 180 seats), 'Large' (between 181 and 250 seats) or 'Jumbo' (more than 250 seats). Keep the `%%timeit` at the beginning of the code cell below to measure the speed of executing your code.

In [ ]:
%%timeit 


### Part C(ii) (2 mark)

Repeat the task from C(i) but using only Numpy operations, to add a new column `size_category_np` to `planes`. Again, use `%%timeit` when running your code. What do you notice in comparison to the result in C(i) in terms of the running time?

In [ ]:
%%timeit


### Part C(iii) (1 mark)

Using the `merge` method from Pandas, add the `size_category` column from `planes` to `flights`. 

### Part C(iv) (1 marks)

Determine for each size category the destination (`dest`) with the most flights. Which carrier uses the most `Large` planes?